# Translator for soccer data from http://www.kicker.de


In [1]:
import os
import six
from google.cloud import translate_v2 as translate
import re
import requests
import pandas as pd
import numpy as np
from time import sleep
from bs4 import BeautifulSoup

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="pmb-key.json"

In [8]:
def translate_text(target, text):
    
    """Translates text into the target language.

    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """
    
    translate_client = translate.Client()

    if isinstance(text, six.binary_type):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)

    return(result["translatedText"])

In [4]:
translation_df = pd.read_csv('../data/pre_translate/' + '2019-20__match_infos_with_grades_and_summary.csv')  

In [6]:
translation_df.head(5)

,date,name_home_team,name_away_team,score_home,score_away,shots_home,shots_away,passes_home,passes_away,misplaced_passes_home,...,distance_away,grade,summary_german,summary_english,coach_home,coach_away,interview_home,interview_away,interview_home_english,interview_away_english
0,"16.08.2019, 20:30",Bayern München,Hertha BSC,2,2,17,6,661,282,79,...,119.19,"2,0","Unterhaltsame Partie, Hertha wehrte sich gut u...","Unterhaltsame Partie, Hertha wehrte sich gut u...",Niko Kovac,Ante Covic,Wir haben die Dominanz und Chancen gehabt. Die...,"Wir wussten von vornherein, dass wir einige gu...",TODO,TODO
1,"17.08.2019, 15:30",Borussia Dortmund,FC Augsburg,5,1,22,5,886,246,68,...,113.09,"2,0","Zwar einseitig, angesichts der Dortmunder Spie...","Zwar einseitig, angesichts der Dortmunder Spie...",Lucien Favre,Martin Schmidt,Wir waren sehr früh überrascht über das 0:1 na...,Nach der Führung haben wir umgehend das 1:1 ka...,TODO,TODO
2,"17.08.2019, 15:30",Bayer 04 Leverkusen,SC Paderborn 07,3,2,13,9,763,267,103,...,123.08,"1,5",Hochintensive Partie mit zahlreichen Chancen v...,Hochintensive Partie mit zahlreichen Chancen v...,Peter Bosz,Steffen Baumgart,"Mit dem Ergebnis bin ich sehr zufrieden, mit d...",Die Jungs haben einen guten Job gemacht. Wicht...,TODO,TODO
3,"17.08.2019, 15:30",VfL Wolfsburg,1. FC Köln,2,1,15,11,377,411,96,...,111.96,"4,0","Wenig Torraumszenen, dafür viel Kampf im Mitte...","Wenig Torraumszenen, dafür viel Kampf im Mitte...",Oliver Glasner,Achim Beierlorzer,"Wir sind sehr zufrieden, dass wir das Auftakts...",Vom Ergebnis bin ich enttäuscht. Ich habe ein ...,TODO,TODO
4,"17.08.2019, 15:30",Werder Bremen,Fortuna Düsseldorf,1,3,24,12,639,308,81,...,117.77,"3,0",Über weite Strecken intensive und spannende Pa...,Über weite Strecken intensive und spannende Pa...,Florian Kohfeldt,Friedhelm Funkel,Wir hatten enorm viel Spielkontrolle in den er...,"Natürlich sind wir sehr, sehr glücklich, dass ...",TODO,TODO


<div style='background:#75CF63; padding: 15px; border: thin solid black; border-radius: 5px; text-align: center'>
translate match summary

In [9]:
translation_df['summary_english'] = translation_df['summary_german'].apply(lambda x: translate_text('en', x))
display(translation_df)

,date,name_home_team,name_away_team,score_home,score_away,shots_home,shots_away,passes_home,passes_away,misplaced_passes_home,...,distance_away,grade,summary_german,summary_english,coach_home,coach_away,interview_home,interview_away,interview_home_english,interview_away_english
0,"16.08.2019, 20:30",Bayern München,Hertha BSC,2,2,17,6,661,282,79,...,119.19,"2,0","Unterhaltsame Partie, Hertha wehrte sich gut u...","Entertaining game, Hertha resisted well and us...",Niko Kovac,Ante Covic,Wir haben die Dominanz und Chancen gehabt. Die...,"Wir wussten von vornherein, dass wir einige gu...",TODO,TODO
1,"17.08.2019, 15:30",Borussia Dortmund,FC Augsburg,5,1,22,5,886,246,68,...,113.09,"2,0","Zwar einseitig, angesichts der Dortmunder Spie...","One-sided, but very entertaining in view of Do...",Lucien Favre,Martin Schmidt,Wir waren sehr früh überrascht über das 0:1 na...,Nach der Führung haben wir umgehend das 1:1 ka...,TODO,TODO
2,"17.08.2019, 15:30",Bayer 04 Leverkusen,SC Paderborn 07,3,2,13,9,763,267,103,...,123.08,"1,5",Hochintensive Partie mit zahlreichen Chancen v...,Highly intense game with numerous chances befo...,Peter Bosz,Steffen Baumgart,"Mit dem Ergebnis bin ich sehr zufrieden, mit d...",Die Jungs haben einen guten Job gemacht. Wicht...,TODO,TODO
3,"17.08.2019, 15:30",VfL Wolfsburg,1. FC Köln,2,1,15,11,377,411,96,...,111.96,"4,0","Wenig Torraumszenen, dafür viel Kampf im Mitte...","Few goal area scenes, but a lot of fighting in...",Oliver Glasner,Achim Beierlorzer,"Wir sind sehr zufrieden, dass wir das Auftakts...",Vom Ergebnis bin ich enttäuscht. Ich habe ein ...,TODO,TODO
4,"17.08.2019, 15:30",Werder Bremen,Fortuna Düsseldorf,1,3,24,12,639,308,81,...,117.77,"3,0",Über weite Strecken intensive und spannende Pa...,"Over long stretches, an intense and exciting g...",Florian Kohfeldt,Friedhelm Funkel,Wir hatten enorm viel Spielkontrolle in den er...,"Natürlich sind wir sehr, sehr glücklich, dass ...",TODO,TODO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,"27.06.2020, 15:30",Eintracht Frankfurt,SC Paderborn 07,3,2,13,17,554,353,82,...,111.72,"4,0",Viele Unzulänglichkeiten bei beiden Mannschaft...,"Many shortcomings in both teams, but five goal...",Adolf Hütter,Steffen Baumgart,"Es hat jeder gesehen, dass wir uns richtig sch...","Wir waren sehr gut im Spiel, die Jungs haben a...",TODO,TODO
302,"27.06.2020, 15:30",Werder Bremen,1. FC Köln,6,1,18,6,476,415,78,...,108.46,"2,0","Bis zur Bremer Führung ausgeglichene, erst dan...","Even before the Bremen leadership, the game wa...",Florian Kohfeldt,Markus Gisdol,Wir haben heute unter Druck eine tolle Leistun...,"Ich kann sagen, dass die ganze Saison nicht ei...",TODO,TODO
303,"27.06.2020, 15:30",SC Freiburg,FC Schalke 04,4,0,15,22,489,367,64,...,114.38,"2,5",Vor allem dank der Freiburger Spielfreude eine...,"An entertaining game with occasional lengths, ...",Christian Streich,David Wagner,Es war ein umkämpftes Spiel. Wir haben es in d...,Es war am Ende eine verdiente Niederlage. Auch...,TODO,TODO
304,"27.06.2020, 15:30",FC Augsburg,RB Leipzig,1,2,5,14,305,586,82,...,109.82,"3,0","Leipzig bot über weite Strecken guten Fußball,...",Leipzig offered good football for long stretch...,Heiko Herrlich,Julian Nagelsmann,"Ich bin froh, dass wir heute noch einmal eine ...",Insgesamt haben wir ein gutes Spiel gemacht un...,TODO,TODO


<div style='background:#75CF63; padding: 15px; border: thin solid black; border-radius: 5px; text-align: center'>
translate home coach interviews

In [11]:
translation_df['interview_home_english'] = translation_df['interview_home'].apply(lambda x: translate_text('en', x))
display(translation_df)

,date,name_home_team,name_away_team,score_home,score_away,shots_home,shots_away,passes_home,passes_away,misplaced_passes_home,...,distance_away,grade,summary_german,summary_english,coach_home,coach_away,interview_home,interview_away,interview_home_english,interview_away_english
0,"16.08.2019, 20:30",Bayern München,Hertha BSC,2,2,17,6,661,282,79,...,119.19,"2,0","Unterhaltsame Partie, Hertha wehrte sich gut u...","Entertaining game, Hertha resisted well and us...",Niko Kovac,Ante Covic,Wir haben die Dominanz und Chancen gehabt. Die...,"Wir wussten von vornherein, dass wir einige gu...",We had the dominance and the chances. The team...,TODO
1,"17.08.2019, 15:30",Borussia Dortmund,FC Augsburg,5,1,22,5,886,246,68,...,113.09,"2,0","Zwar einseitig, angesichts der Dortmunder Spie...","One-sided, but very entertaining in view of Do...",Lucien Favre,Martin Schmidt,Wir waren sehr früh überrascht über das 0:1 na...,Nach der Führung haben wir umgehend das 1:1 ka...,We were surprised very early about the 0: 1 af...,TODO
2,"17.08.2019, 15:30",Bayer 04 Leverkusen,SC Paderborn 07,3,2,13,9,763,267,103,...,123.08,"1,5",Hochintensive Partie mit zahlreichen Chancen v...,Highly intense game with numerous chances befo...,Peter Bosz,Steffen Baumgart,"Mit dem Ergebnis bin ich sehr zufrieden, mit d...",Die Jungs haben einen guten Job gemacht. Wicht...,"I&#39;m very satisfied with the result, but no...",TODO
3,"17.08.2019, 15:30",VfL Wolfsburg,1. FC Köln,2,1,15,11,377,411,96,...,111.96,"4,0","Wenig Torraumszenen, dafür viel Kampf im Mitte...","Few goal area scenes, but a lot of fighting in...",Oliver Glasner,Achim Beierlorzer,"Wir sind sehr zufrieden, dass wir das Auftakts...",Vom Ergebnis bin ich enttäuscht. Ich habe ein ...,We are very satisfied that we were able to win...,TODO
4,"17.08.2019, 15:30",Werder Bremen,Fortuna Düsseldorf,1,3,24,12,639,308,81,...,117.77,"3,0",Über weite Strecken intensive und spannende Pa...,"Over long stretches, an intense and exciting g...",Florian Kohfeldt,Friedhelm Funkel,Wir hatten enorm viel Spielkontrolle in den er...,"Natürlich sind wir sehr, sehr glücklich, dass ...",We had a lot of control over the game in the f...,TODO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,"27.06.2020, 15:30",Eintracht Frankfurt,SC Paderborn 07,3,2,13,17,554,353,82,...,111.72,"4,0",Viele Unzulänglichkeiten bei beiden Mannschaft...,"Many shortcomings in both teams, but five goal...",Adolf Hütter,Steffen Baumgart,"Es hat jeder gesehen, dass wir uns richtig sch...","Wir waren sehr gut im Spiel, die Jungs haben a...",Everyone saw that we had a really hard time. C...,TODO
302,"27.06.2020, 15:30",Werder Bremen,1. FC Köln,6,1,18,6,476,415,78,...,108.46,"2,0","Bis zur Bremer Führung ausgeglichene, erst dan...","Even before the Bremen leadership, the game wa...",Florian Kohfeldt,Markus Gisdol,Wir haben heute unter Druck eine tolle Leistun...,"Ich kann sagen, dass die ganze Saison nicht ei...",We put in a great performance today under pres...,TODO
303,"27.06.2020, 15:30",SC Freiburg,FC Schalke 04,4,0,15,22,489,367,64,...,114.38,"2,5",Vor allem dank der Freiburger Spielfreude eine...,"An entertaining game with occasional lengths, ...",Christian Streich,David Wagner,Es war ein umkämpftes Spiel. Wir haben es in d...,Es war am Ende eine verdiente Niederlage. Auch...,It was a competitive game. We did very well in...,TODO
304,"27.06.2020, 15:30",FC Augsburg,RB Leipzig,1,2,5,14,305,586,82,...,109.82,"3,0","Leipzig bot über weite Strecken guten Fußball,...",Leipzig offered good football for long stretch...,Heiko Herrlich,Julian Nagelsmann,"Ich bin froh, dass wir heute noch einmal eine ...",Insgesamt haben wir ein gutes Spiel gemacht un...,I am happy that we put in another good perform...,TODO


<div style='background:#75CF63; padding: 15px; border: thin solid black; border-radius: 5px; text-align: center'>
translate home coach interviews

In [12]:
translation_df['interview_away_english'] = translation_df['interview_away'].apply(lambda x: translate_text('en', x))

In [21]:
translation_df.to_csv(r'../data/translated/2019-20__match_infos_with_grades_and_summary_translated.csv', index = False, encoding='utf-8-sig')

PermissionError: [Errno 13] Permission denied: '../data/translated/2019-20__match_infos_with_grades_and_summary_translated.csv'

<div style='background:#75CF63; padding: 15px; border: thin solid black; border-radius: 5px; text-align: center'>
additional cleanup

In [24]:
cleanup_df = pd.read_csv('../data/translated/2019-20__match_infos_with_grades_and_summary_translated.csv')

In [39]:

cleanup_df['interview_home_english'] = cleanup_df['interview_home_english'].apply(lambda x: x.replace('&#39;', '\''))
cleanup_df['interview_away_english'] = cleanup_df['interview_away_english'].apply(lambda x: x.replace('&#39;', '\''))
cleanup_df['summary_english'] = cleanup_df['summary_english'].apply(lambda x: x.replace('&#39;', '\''))

In [43]:
cleanup_df.head(50)

,date,name_home_team,name_away_team,score_home,score_away,shots_home,shots_away,passes_home,passes_away,misplaced_passes_home,...,distance_away,grade,summary_german,summary_english,coach_home,coach_away,interview_home,interview_away,interview_home_english,interview_away_english
0,"16.08.2019, 20:30",Bayern München,Hertha BSC,2,2,17,6,661,282,79,...,119.19,"2,0","Unterhaltsame Partie, Hertha wehrte sich gut u...","Entertaining game, Hertha resisted well and us...",Niko Kovac,Ante Covic,Wir haben die Dominanz und Chancen gehabt. Die...,"Wir wussten von vornherein, dass wir einige gu...",We had the dominance and the chances. The team...,We knew from the start that we had to have som...
1,"17.08.2019, 15:30",Borussia Dortmund,FC Augsburg,5,1,22,5,886,246,68,...,113.09,"2,0","Zwar einseitig, angesichts der Dortmunder Spie...","One-sided, but very entertaining in view of Do...",Lucien Favre,Martin Schmidt,Wir waren sehr früh überrascht über das 0:1 na...,Nach der Führung haben wir umgehend das 1:1 ka...,We were surprised very early about the 0: 1 af...,After the tour we immediately got the 1: 1 and...
2,"17.08.2019, 15:30",Bayer 04 Leverkusen,SC Paderborn 07,3,2,13,9,763,267,103,...,123.08,"1,5",Hochintensive Partie mit zahlreichen Chancen v...,Highly intense game with numerous chances befo...,Peter Bosz,Steffen Baumgart,"Mit dem Ergebnis bin ich sehr zufrieden, mit d...",Die Jungs haben einen guten Job gemacht. Wicht...,"I'm very satisfied with the result, but not ye...",The guys did a good job. It was important for ...
3,"17.08.2019, 15:30",VfL Wolfsburg,1. FC Köln,2,1,15,11,377,411,96,...,111.96,"4,0","Wenig Torraumszenen, dafür viel Kampf im Mitte...","Few goal area scenes, but a lot of fighting in...",Oliver Glasner,Achim Beierlorzer,"Wir sind sehr zufrieden, dass wir das Auftakts...",Vom Ergebnis bin ich enttäuscht. Ich habe ein ...,We are very satisfied that we were able to win...,I am disappointed with the result. I saw my te...
4,"17.08.2019, 15:30",Werder Bremen,Fortuna Düsseldorf,1,3,24,12,639,308,81,...,117.77,"3,0",Über weite Strecken intensive und spannende Pa...,"Over long stretches, an intense and exciting g...",Florian Kohfeldt,Friedhelm Funkel,Wir hatten enorm viel Spielkontrolle in den er...,"Natürlich sind wir sehr, sehr glücklich, dass ...",We had a lot of control over the game in the f...,"Of course we are very, very happy that we won ..."
5,"17.08.2019, 15:30",SC Freiburg,1. FSV Mainz 05,3,0,19,19,400,372,88,...,107.96,"2,5",Zwischen einem flotten Beginn und einer torrei...,There were also a few lengths between a brisk ...,Christian Streich,Sandro Schwarz,"Es war sehr ausgeglichen, beide Teams können i...",Es war ein kurioses Spiel. Wir hatten alles im...,"It was very even, both teams can take the lead...",It was a strange game. We had everything under...
6,"17.08.2019, 18:30",Bor. Mönchengladbach,FC Schalke 04,0,0,16,8,519,351,101,...,115.64,"3,5","Rassiges Duell, dem jedoch die spielerischen ...","Classy duel, which however lacked the playful ...",Marco Rose,David Wagner,"Die Leistung war in Ordnung, wir haben wieder ...",Die beiden Mannschaften haben sich gegenseitig...,"The performance was okay, we took another smal...",The two teams fought and pressed each other. T...
7,"18.08.2019, 15:30",Eintracht Frankfurt,TSG Hoffenheim,1,0,15,9,250,567,92,...,109.15,"3,5",Insgesamt recht zäh und mit Auszeiten auf beid...,All in all quite tough and with time-outs on b...,Adolf Hütter,Alfred Schreuder,Es ist ein ganz wichtiger Sieg. Wir haben eine...,Glückwunsch an Frankfurt. Am Ende ist es verdi...,It's a very important victory. We had a strong...,Congratulations to Frankfurt. In the end it is...
8,"18.08.2019, 18:00",1. FC Union Berlin,RB Leipzig,0,4,13,14,325,605,94,...,113.84,"2,5","Einseitige Partie, Leipzig mit gepflegtem Fußb...","One-sided game, Leipzig with well-groomed foot...",Urs Fischer,Julian Nagelsmann,Wir haben in der ersten Halbzeit zu viele Gesc...,Es hätte auf jeden fall schlechter kommen könn...,We 

In [45]:
cleanup_df.to_csv(r'../data/final/2019-20__match_infos_with_grades_and_summary_translated_cleaned.csv', index = False, encoding='utf-8-sig')